
---

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

from sklearn.model_selection import train_test_split

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [2]:
# Load English data
english_sentences = helper.load_data('small_vocab_en.txt')
# Load French data
french_sentences = helper.load_data('small_vocab_fr.txt')

print('Dataset Loaded')
print(type(english_sentences))

Dataset Loaded
<class 'list'>


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [3]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [4]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [5]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # Implement
    
    # Create the tokeninzer
    t = Tokenizer()
    # Create dictionary mapping words (str) to their rank/index (int)
    t.fit_on_texts(x)
    # Use the tokenizer to tokenize the text
    text_sequences = t.texts_to_sequences(x)
    return text_sequences, t

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [6]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    
    # If length equals None, set it to be the length of the longest sequence in x
    if length == None:
        length = len(max(x, key=len))
        
    # Use Keras's pad_sequences to pad the sequences with 0's
    padded_sequences = pad_sequences(sequences=x, maxlen=length, padding='post', value=0)
    
    return padded_sequences

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [7]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 345


### Extension - Split the data
Use `sklearn.model_selection.train_test_split` to split the data into training and test sets.

Note: `train_test_split` should be perferred on X and y data together, not seperately.

For example,

`train_test_split(X, y, test_size = 0.2, random_state = 42)`

Do not split the data seperately (as seen as the commented out code in the below cell).

In [8]:
### Split the unprocessed data into training and test sets using Sklearn ###

english_sentences_train, english_sentences_test, french_sentences_train, french_sentences_test =\
    train_test_split(english_sentences, french_sentences, test_size=0.2, random_state=42)
    
    


In [9]:
# Calculate the training set stats

english_train_words_counter = collections.Counter([word for sentence in english_sentences_train for word in sentence.split()])
french_train_words_counter = collections.Counter([word for sentence in french_sentences_train for word in sentence.split()])

print('{} English words in the training set.'.format(len([word for sentence in english_sentences_train for word in sentence.split()])))
print('{} unique English words in the training set.'.format(len(english_train_words_counter)))
print('10 Most common words in the English training dataset:')
print('"' + '" "'.join(list(zip(*english_train_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words in the training set.'.format(len([word for sentence in french_sentences_train for word in sentence.split()])))
print('{} unique French words in the training set.'.format(len(french_train_words_counter)))
print('10 Most common words in the French training dataset:')
print('"' + '" "'.join(list(zip(*french_train_words_counter.most_common(10)))[0]) + '"')

1458806 English words in the training set.
227 unique English words in the training set.
10 Most common words in the English training dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1568964 French words in the training set.
354 unique French words in the training set.
10 Most common words in the French training dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [10]:
# Calculate the test set stats

english_test_words_counter = collections.Counter([word for sentence in english_sentences_test for word in sentence.split()])
french_test_words_counter = collections.Counter([word for sentence in french_sentences_test for word in sentence.split()])

print('{} English words in the test set.'.format(len([word for sentence in english_sentences_test for word in sentence.split()])))
print('{} unique English words in the test set.'.format(len(english_test_words_counter)))
print('10 Most common words in the English test dataset:')
print('"' + '" "'.join(list(zip(*english_test_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words in the test set.'.format(len([word for sentence in french_sentences_test for word in sentence.split()])))
print('{} unique French words in the test set.'.format(len(french_test_words_counter)))
print('10 Most common words in the French test dataset:')
print('"' + '" "'.join(list(zip(*french_test_words_counter.most_common(10)))[0]) + '"')

364444 English words in the test set.
227 unique English words in the test set.
10 Most common words in the English test dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

392331 French words in the test set.
338 unique French words in the test set.
10 Most common words in the French test dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


### Preprocess the training data

Use `preprocess` to process the training split of the data. 

In [11]:
# Preprocess the training split of the data

preproc_english_sentences_train, preproc_french_sentences_train, english_tokenizer_train, french_tokenizer_train =\
    preprocess(english_sentences_train, french_sentences_train)
    
max_english_sequence_length_train = preproc_english_sentences_train.shape[1]
max_french_sequence_length_train = preproc_french_sentences_train.shape[1]
english_vocab_size_train = len(english_tokenizer_train.word_index)
french_vocab_size_train = len(french_tokenizer_train.word_index)

print('Training Data Preprocessed')
print("Max English train dataset sentence length:", max_english_sequence_length_train)
print("Max French train dataset sentence length:", max_french_sequence_length_train)
print("English train dataset vocabulary size:", english_vocab_size_train)
print("French train dataset vocabulary size:", french_vocab_size_train)

Training Data Preprocessed
Max English train dataset sentence length: 15
Max French train dataset sentence length: 21
English train dataset vocabulary size: 199
French train dataset vocabulary size: 344


### Preprocess the training data

Use `preprocess` to process the test split of the data. 

In [12]:
# Preprocess the test split of the data 

preproc_english_sentences_test, preproc_french_sentences_test, english_tokenizer_test, french_tokenizer_test =\
    preprocess(english_sentences_test, french_sentences_test)
    
max_english_sequence_length_test = preproc_english_sentences_test.shape[1]
max_french_sequence_length_test = preproc_french_sentences_test.shape[1]
english_vocab_size_test = len(english_tokenizer_test.word_index)
french_vocab_size_test = len(french_tokenizer_test.word_index)

print('Test Data Preprocessed')
print('Max English test dataset sentence length:', max_english_sequence_length_test)
print('Max French test dataset sentence length:', max_french_sequence_length_test)
print('English test datset vocab size:', english_vocab_size_test)
print('French test dataset vocab size', french_vocab_size_test)
print(preproc_french_sentences_test.shape)

Test Data Preprocessed
Max English test dataset sentence length: 15
Max French test dataset sentence length: 20
English test datset vocab size: 199
French test dataset vocab size 327
(27573, 20, 1)


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [13]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [14]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    
    # Input layer
    input_layer = Input(shape=input_shape[1:])
    
    # GRU Layer
    x = GRU(output_sequence_length, return_sequences=True)(input_layer)
    
    # Fully-connected Layer
    x = Dense(french_vocab_size*4, activation='relu')(x)
    
    # Output Layer
    output_layer = Dense(french_vocab_size, activation='softmax')(x)
    
    # Create model
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compile the model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr = 0.01),
                  metrics=['accuracy'])
    return model


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the model on the training set
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size + 1,
    french_vocab_size + 1)

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=528, epochs=20, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/20
209/209 [==============================] - 122s 582ms/step - loss: 1.8143 - accuracy: 0.5629 - val_loss: 1.3701 - val_accuracy: 0.6205
Epoch 2/20
209/209 [==============================] - 121s 579ms/step - loss: 1.2667 - accuracy: 0.6428 - val_loss: 1.1899 - val_accuracy: 0.6581
Epoch 3/20
209/209 [==============================] - 124s 593ms/step - loss: 1.1387 - accuracy: 0.6682 - val_loss: 1.0893 - val_accuracy: 0.6725
Epoch 4/20
209/209 [==============================] - 123s 586ms/step - loss: 1.0625 - accuracy: 0.6809 - val_loss: 1.0448 - val_accuracy: 0.6845
Epoch 5/20
209/209 [==============================] - 123s 587ms/step - loss: 1.0145 - accuracy: 0.6879 - val_loss: 0.9822 - val_accuracy: 0.6950
Epoch 6/20
209/209 [==============================] - 122s 585ms/step - loss: 0.9780 - accuracy: 0.6946 - val_loss: 1.0082 - val_accuracy: 0.6831
Epoch 7/20
209/209 [==============================] - 123s 589ms/step - loss: 0.9584 - accuracy: 0.6973 - val_loss: 0.9226 -

### Train the simple model on the training data set

Use the same model as above on the training data split to see how it performs on the training split of the data.

In [15]:
### Try the simple_model on the training split of the dataset

def simple_model_split(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    
    # Input layer
    input_layer = Input(shape=input_shape[1:])
    
    # GRU Layer
    x = GRU(256, return_sequences=True)(input_layer)
    
    # Fully-connected Layer
    x = Dense(french_vocab_size*4, activation='relu')(x)
    
    # Output Layer
    output_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    
    # Create model
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compile the model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr = 0.01),
                  metrics=['accuracy'])
    return model

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences_train, max_french_sequence_length_train)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

# Train the model on the training set
simple_rnn_model_split = simple_model_split(
    tmp_x.shape,
    max_french_sequence_length_train,
    english_vocab_size_train + 1,
    french_vocab_size_train + 1)

simple_rnn_model_split.fit(tmp_x, preproc_french_sentences_train, batch_size=200, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model_split.predict(tmp_x[:1])[0], french_tokenizer_train))

Epoch 1/10
442/442 [==============================] - 193s 437ms/step - loss: 1.2439 - accuracy: 0.6520 - val_loss: 0.9766 - val_accuracy: 0.6846
Epoch 2/10
442/442 [==============================] - 179s 405ms/step - loss: 0.8285 - accuracy: 0.7299 - val_loss: 0.7534 - val_accuracy: 0.7524
Epoch 3/10
442/442 [==============================] - 183s 414ms/step - loss: 0.7300 - accuracy: 0.7547 - val_loss: 0.7007 - val_accuracy: 0.7589
Epoch 4/10
442/442 [==============================] - 188s 426ms/step - loss: 0.6680 - accuracy: 0.7714 - val_loss: 0.6421 - val_accuracy: 0.7787
Epoch 5/10
442/442 [==============================] - 186s 421ms/step - loss: 0.6266 - accuracy: 0.7850 - val_loss: 0.5951 - val_accuracy: 0.7951
Epoch 6/10
442/442 [==============================] - 181s 410ms/step - loss: 0.5994 - accuracy: 0.7948 - val_loss: 0.6451 - val_accuracy: 0.7788
Epoch 7/10
442/442 [==============================] - 191s 432ms/step - loss: 0.5679 - accuracy: 0.8032 - val_loss: 0.5806 -

### Test the simple model on the test data set

See how the model trained on the training set data performs on the unseen test data.

In [16]:
### Evaluate the model on the test set ###

# Reshape the test data 
test_x = pad(preproc_english_sentences_test, max_french_sequence_length_test)
test_x = test_x.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

# Rehsape the test label data to fit the model
preproc_french_sentences_test = preproc_french_sentences_test.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

# Evaluate the model on the unseen test data
simple_rnn_model_score = simple_rnn_model_split.evaluate(test_x, preproc_french_sentences_test, verbose=0)

print("Simple model accuracy on unseen test dataset: {0:.2f}%".format(simple_rnn_model_score[1]*100))

Simple model accuracy on unseen test dataset: 39.36%


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [17]:
### Embedding model trained on entire dataset ###

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    #  Implement
    
    # RNN model with embedding layer
    input_layer = Input(shape=input_shape[1:])
    embedding_layer = Embedding(512, english_vocab_size)(input_layer)
    x = GRU(512, return_sequences=True)(embedding_layer)
    x = TimeDistributed(Dense(french_vocab_size*4, activation='relu'))(x)
    output_layer = Dense(french_vocab_size, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compile the model
    learning_rate = 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Test the model


#  Pad the input 
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])

# Train the neural network on the training data
embedding_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size + 1,
    french_vocab_size + 1)
embedding_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embedding_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 450s 4s/step - loss: 7.9308 - accuracy: 0.3199 - val_loss: 5.6051 - val_accuracy: 0.4930
Epoch 2/10
108/108 [==============================] - 452s 4s/step - loss: 1.3273 - accuracy: 0.7267 - val_loss: 0.7896 - val_accuracy: 0.8048
Epoch 3/10
108/108 [==============================] - 447s 4s/step - loss: 0.6796 - accuracy: 0.8321 - val_loss: 0.6047 - val_accuracy: 0.8529
Epoch 4/10
108/108 [==============================] - 453s 4s/step - loss: 0.5416 - accuracy: 0.8702 - val_loss: 0.5009 - val_accuracy: 0.8843
Epoch 5/10
108/108 [==============================] - 450s 4s/step - loss: 0.4695 - accuracy: 0.8917 - val_loss: 0.4478 - val_accuracy: 0.8973
Epoch 6/10
108/108 [==============================] - 458s 4s/step - loss: 0.4311 - accuracy: 0.9026 - val_loss: 0.4332 - val_accuracy: 0.9029
Epoch 7/10
108/108 [==============================] - 492s 5s/step - loss: 0.4046 - accuracy: 0.9102 - val_loss: 0.3964 - val_accuracy: 0.9114

### Train the embedding model on the training data set

Use the same model as above on the training data split to see how it performs on the training split of the data.

In [18]:
### Train the embedding model on the training dataset ###

def embed_model_split(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    
    # RNN model with embedding layer
    input_layer = Input(shape=input_shape[1:])
    embedding_layer = Embedding(512, english_vocab_size)(input_layer)
    x = GRU(512, return_sequences=True)(embedding_layer)
    x = TimeDistributed(Dense(french_vocab_size*4, activation='relu'))(x)
    output_layer = Dense(french_vocab_size, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compile the model
    learning_rate = 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Pad the input 
tmp_x = pad(preproc_english_sentences_train, preproc_french_sentences_train.shape[1])

# Train the neural network on the training data
embedding_model_split_train = embed_model_split(tmp_x.shape,
                                  max_french_sequence_length_train,
                                  english_vocab_size_train + 1,
                                  french_vocab_size_train + 1)

embedding_model_split_train.fit(tmp_x, preproc_french_sentences_train, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embedding_model_split_train.predict(tmp_x[:1])[0], french_tokenizer_train))

Epoch 1/10
87/87 [==============================] - 378s 4s/step - loss: 11.0263 - accuracy: 0.2738 - val_loss: 9.0424 - val_accuracy: 0.4386
Epoch 2/10
87/87 [==============================] - 364s 4s/step - loss: 9.0100 - accuracy: 0.4397 - val_loss: 8.9809 - val_accuracy: 0.4418
Epoch 3/10
87/87 [==============================] - 362s 4s/step - loss: 8.9985 - accuracy: 0.4405 - val_loss: 8.9763 - val_accuracy: 0.4421
Epoch 4/10
87/87 [==============================] - 362s 4s/step - loss: 8.9945 - accuracy: 0.4407 - val_loss: 8.9726 - val_accuracy: 0.4424
Epoch 5/10
87/87 [==============================] - 376s 4s/step - loss: 8.9941 - accuracy: 0.4407 - val_loss: 8.9753 - val_accuracy: 0.4422
Epoch 6/10
87/87 [==============================] - 365s 4s/step - loss: 8.9901 - accuracy: 0.4409 - val_loss: 8.9607 - val_accuracy: 0.4411
Epoch 7/10
87/87 [==============================] - 378s 4s/step - loss: 8.9779 - accuracy: 0.4418 - val_loss: 8.9568 - val_accuracy: 0.4434
Epoch 8/10
8

### Test the embedding model on the test data set

See how the model trained on the training set data performs on the unseen test data.

In [19]:
### Evaluate the embedding model on the unseen test data ###

test_x = pad(preproc_english_sentences_test, max_french_sequence_length_test)
test_x = test_x.reshape((-1, preproc_french_sentences_train.shape[-2]))

embedding_model_score = embedding_model_split_train.evaluate(test_x, preproc_french_sentences_test, verbose=0)

print("Embedding model accuracy on unseen test data: {0:.2f}%".format(embedding_model_score[1]*100))

Embedding model accuracy on unseen test data: 38.70%


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [20]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    
    # Build the bidirectional RNN model
    input_layer = Input(shape=input_shape[1:])
    x = Bidirectional(GRU(512, return_sequences=True, recurrent_dropout=0.2))(input_layer)
    x = Dense(french_vocab_size*2, activation='relu')(x)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compile the model
    learning_rate = 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


# Pad the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# TODO: Train and Print prediction(s)

# Train the model
bidirectional_model = bd_model(tmp_x.shape,
                               max_french_sequence_length,
                               english_vocab_size + 1,
                               french_vocab_size + 1)
# Fit the model
bidirectional_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print out the predictions
print(logits_to_text(bidirectional_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 642s 6s/step - loss: 1.7893 - accuracy: 0.5865 - val_loss: 1.1286 - val_accuracy: 0.6603
Epoch 2/10
108/108 [==============================] - 604s 6s/step - loss: 1.0620 - accuracy: 0.6723 - val_loss: 0.9797 - val_accuracy: 0.6820
Epoch 3/10
108/108 [==============================] - 604s 6s/step - loss: 0.9634 - accuracy: 0.6875 - val_loss: 0.8874 - val_accuracy: 0.7059
Epoch 4/10
108/108 [==============================] - 611s 6s/step - loss: 0.9080 - accuracy: 0.6979 - val_loss: 0.8778 - val_accuracy: 0.6995
Epoch 5/10
108/108 [==============================] - 609s 6s/step - loss: 0.8844 - accuracy: 0.7022 - val_loss: 0.8589 - val_accuracy: 0.7012
Epoch 6/10
108/108 [==============================] - 613s 6s/step - loss: 0.8420 - accuracy: 0.7149 - val_loss: 0.7492 - val_accuracy: 0.7447
Epoch 7/10
108/108 [==============================] - 618s 6s/step - loss: 0.8030 - accuracy: 0.7284 - val_loss: 0.7812 - val_accuracy: 0.7259

### Train the bidirectional model on the training data set

Use the same model as above on the training data split to see how it performs.

In [21]:
### Train the bidirectional model on the training data set ###

def bd_model_split(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # Build the bidirectional RNN model
    input_layer = Input(shape=input_shape[1:])
    x = Bidirectional(GRU(512, return_sequences=True, recurrent_dropout=0.2))(input_layer)
    x = Dense(french_vocab_size, activation='relu')(x)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compile the model
    learning_rate = 0.001
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

# Pad the input
tmp_x = pad(preproc_english_sentences_train, max_french_sequence_length_train)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

# Train the model
bidirectional_model_train = bd_model_split(tmp_x.shape,
                               max_french_sequence_length_train,
                               english_vocab_size_train + 1,
                               french_vocab_size_train + 1)
# Fit the model
bidirectional_model_train.fit(tmp_x, preproc_french_sentences_train, batch_size=1024, epochs=10, validation_split=0.2)

# Print out the predictions
print(logits_to_text(bidirectional_model_train.predict(tmp_x[:1])[0], french_tokenizer_train))

Epoch 1/10
87/87 [==============================] - 449s 5s/step - loss: 2.0372 - accuracy: 0.5559 - val_loss: 1.4171 - val_accuracy: 0.6170
Epoch 2/10
87/87 [==============================] - 450s 5s/step - loss: 1.2890 - accuracy: 0.6407 - val_loss: 1.1842 - val_accuracy: 0.6554
Epoch 3/10
87/87 [==============================] - 448s 5s/step - loss: 1.1304 - accuracy: 0.6671 - val_loss: 1.0727 - val_accuracy: 0.6733
Epoch 4/10
87/87 [==============================] - 448s 5s/step - loss: 1.0410 - accuracy: 0.6831 - val_loss: 0.9840 - val_accuracy: 0.6969
Epoch 5/10
87/87 [==============================] - 450s 5s/step - loss: 0.9744 - accuracy: 0.6950 - val_loss: 0.9170 - val_accuracy: 0.7125
Epoch 6/10
87/87 [==============================] - 453s 5s/step - loss: 0.9169 - accuracy: 0.7094 - val_loss: 0.8842 - val_accuracy: 0.7214
Epoch 7/10
87/87 [==============================] - 457s 5s/step - loss: 0.8662 - accuracy: 0.7246 - val_loss: 0.8149 - val_accuracy: 0.7412
Epoch 8/10
87

### Test the bidirectional model on the test data set

See how the model trained on the training set data performs on the unseen test data.

In [22]:
### Evaluate the bidirectional model on the unseen test data set ###

test_x = pad(preproc_english_sentences_test, max_french_sequence_length_test)
test_x = test_x.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

bidirectional_model_score = bidirectional_model_train.evaluate(test_x, preproc_french_sentences_test, verbose=0)

print("Bidirectional model accuracy on test data: {0:.2f}%".format(bidirectional_model_score[1]*100))

Bidirectional model accuracy on test data: 22.15%


### Model 4: Encoder-Decoder 
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [23]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    #  Implement
    
    # Build the encoder-decoder RNN model
    input_layer= Input(shape=input_shape[1:])
    encoder = GRU(256, recurrent_dropout=0.2)(input_layer)
    repeat_vector = RepeatVector(output_sequence_length)(encoder)
    decoder = GRU(256, return_sequences=True, recurrent_dropout=0.2)(repeat_vector)
    dense_layer = Dense(french_vocab_size, activation='relu')(decoder)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compile the model
    learning_rate = 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    
    return model


# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the model
encoder_decoder_model = encdec_model(tmp_x.shape,
                                     max_french_sequence_length,
                                     english_vocab_size + 1,
                                     french_vocab_size + 1)
encoder_decoder_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print the prediction(s)
print(logits_to_text(encoder_decoder_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 258s 2s/step - loss: 2.5209 - accuracy: 0.4606 - val_loss: 1.9887 - val_accuracy: 0.5192
Epoch 2/10
108/108 [==============================] - 256s 2s/step - loss: 1.8524 - accuracy: 0.5379 - val_loss: 1.6910 - val_accuracy: 0.5763
Epoch 3/10
108/108 [==============================] - 255s 2s/step - loss: 1.6509 - accuracy: 0.5727 - val_loss: 1.5752 - val_accuracy: 0.5807
Epoch 4/10
108/108 [==============================] - 257s 2s/step - loss: 1.4952 - accuracy: 0.5947 - val_loss: 1.3875 - val_accuracy: 0.6189
Epoch 5/10
108/108 [==============================] - 258s 2s/step - loss: 1.4258 - accuracy: 0.6080 - val_loss: 1.3449 - val_accuracy: 0.6266
Epoch 6/10
108/108 [==============================] - 257s 2s/step - loss: 1.3790 - accuracy: 0.6176 - val_loss: 1.3220 - val_accuracy: 0.6342
Epoch 7/10
108/108 [==============================] - 259s 2s/step - loss: 1.3247 - accuracy: 0.6305 - val_loss: 1.2567 - val_accuracy: 0.6530

### Train the encoder-decoder model on the training dataset

Use the same enc-dec model as above but train it on the training data split of the initial dataset.

In [24]:
### Train the encoder-decoder model on the training split of the dataset ###

def encdec_model_train(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # Build the encoder-decoder RNN model
    input_layer= Input(shape=input_shape[1:])
    encoder = GRU(256, recurrent_dropout=0.2)(input_layer)
    repeat_vector = RepeatVector(output_sequence_length)(encoder)
    decoder = GRU(256, return_sequences=True, recurrent_dropout=0.2)(repeat_vector)
    dense_layer = Dense(french_vocab_size, activation='relu')(decoder)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compile the model
    learning_rate = 0.01
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    
    return model

# Reshape the input
tmp_x = pad(preproc_english_sentences_train, max_french_sequence_length_train)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

# Train the model
encoder_decoder_model_train = encdec_model_train(tmp_x.shape,
                                     max_french_sequence_length_train,
                                     english_vocab_size_train + 1,
                                     french_vocab_size_train + 1)
encoder_decoder_model_train.fit(tmp_x, preproc_french_sentences_train, batch_size=1024, epochs=10, validation_split=0.2)

# Print the prediction(s)
print(logits_to_text(encoder_decoder_model_train.predict(tmp_x[:1])[0], french_tokenizer_train))

Epoch 1/10
87/87 [==============================] - 206s 2s/step - loss: 2.6060 - accuracy: 0.4585 - val_loss: 2.0334 - val_accuracy: 0.5162
Epoch 2/10
87/87 [==============================] - 205s 2s/step - loss: 1.8479 - accuracy: 0.5388 - val_loss: 1.7309 - val_accuracy: 0.5492
Epoch 3/10
87/87 [==============================] - 206s 2s/step - loss: 1.6268 - accuracy: 0.5741 - val_loss: 1.5409 - val_accuracy: 0.5911
Epoch 4/10
87/87 [==============================] - 205s 2s/step - loss: 1.5115 - accuracy: 0.5917 - val_loss: 1.4505 - val_accuracy: 0.6013
Epoch 5/10
87/87 [==============================] - 206s 2s/step - loss: 1.4261 - accuracy: 0.6066 - val_loss: 1.3557 - val_accuracy: 0.6242
Epoch 6/10
87/87 [==============================] - 205s 2s/step - loss: 1.3904 - accuracy: 0.6112 - val_loss: 1.3271 - val_accuracy: 0.6247
Epoch 7/10
87/87 [==============================] - 205s 2s/step - loss: 1.3388 - accuracy: 0.6229 - val_loss: 1.2819 - val_accuracy: 0.6379
Epoch 8/10
87

### Evaluate the enc-dec model on the test set

Use the unseen test data set to evaluate the enc-dec model trained on the training data.

In [25]:
### Evaluate the encoder-decoder model on the unseen test dataset ###

# Shape the data to fit
test_x = pad(preproc_english_sentences_test, max_french_sequence_length_test)
test_x = test_x.reshape((-1, preproc_french_sentences_train.shape[-2], 1))
preproc_french_sentences_test = preproc_french_sentences_test.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

print(test_x.shape)
print(preproc_french_sentences_train.shape)
print(preproc_french_sentences_test.shape)

# Evaluate the model
encoder_decoder_model_score = encoder_decoder_model_train.evaluate(test_x, preproc_french_sentences_test, verbose=0)

print("Encoder-decoder model accuracy on the unseen test data: {0:.2f}%".format(encoder_decoder_model_score[1]*100))

(26260, 21, 1)
(110288, 21, 1)
(26260, 21, 1)
Encoder-decoder model accuracy on the unseen test data: 23.06%


### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [26]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    #  Implement
    
    # Final model 2 - simplified version of final model 1
    # Added Dense layer with relu activation after first GRU layer instead of after 2nd GRU layer
    
    input_layer = Input(shape=input_shape[1:])
    embedding_layer = Embedding(english_vocab_size*2, english_vocab_size)(input_layer)
    x = Bidirectional(GRU(256))(embedding_layer)
    x = Dense(256, activation='relu')(x)
    x = RepeatVector(output_sequence_length)(x)
    x = Bidirectional(GRU(256, return_sequences=True))(x)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(x)
    
    
    ### Final model 1 ###
    
    # Build RNN model with embedding, encoder-decoder and bidirectional layers
    #input_layer = Input(shape=input_shape[1:])
    #embedding_layer = Embedding(750, english_vocab_size)(input_layer)
    #encoder = GRU(750)(embedding_layer)
    #repeat_vector = RepeatVector(output_sequence_length)(encoder)
    #decoder = Bidirectional(GRU(750, return_sequences=True))(repeat_vector)
    #dense_layer = TimeDistributed(Dense(french_vocab_size*2, activation='relu'))(decoder)
    #output_layer = Dense(french_vocab_size, activation='softmax')(dense_layer)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    # Compile the model
    learning_rate = 0.005
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    return model



print('Final Model Loaded')

Final Model Loaded


## Prediction (IMPLEMENTATION)

In [27]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    
    
    # Reshape the input
    x = pad(x, y.shape[1])
    
    # Define final_model
    final_model = model_final(x.shape, 
                              y.shape[1],
                              len(x_tk.word_index) + 1,
                              len(y_tk.word_index) + 1)
    # Fit the final model
    final_model.fit(x, y, batch_size=1024, epochs=10, validation_split=0.2)
    
    model = final_model

    
    
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Epoch 1/10
108/108 [==============================] - 471s 4s/step - loss: 2.1109 - accuracy: 0.5237 - val_loss: 1.3870 - val_accuracy: 0.6271
Epoch 2/10
108/108 [==============================] - 467s 4s/step - loss: 1.0996 - accuracy: 0.6974 - val_loss: 0.8795 - val_accuracy: 0.7462
Epoch 3/10
108/108 [==============================] - 469s 4s/step - loss: 0.7422 - accuracy: 0.7805 - val_loss: 0.6502 - val_accuracy: 0.8045
Epoch 4/10
108/108 [==============================] - 467s 4s/step - loss: 0.5230 - accuracy: 0.8429 - val_loss: 0.3883 - val_accuracy: 0.8848
Epoch 5/10
108/108 [==============================] - 468s 4s/step - loss: 0.3172 - accuracy: 0.9086 - val_loss: 0.2426 - val_accuracy: 0.9344
Epoch 6/10
108/108 [==============================] - 469s 4s/step - loss: 0.2140 - accuracy: 0.9404 - val_loss: 0.1962 - val_accuracy: 0.9446
Epoch 7/10
108/108 [==============================] - 468s 4s/step - loss: 0.1781 - accuracy: 0.9494 - val_loss: 0.1999 - val_accuracy: 0.9422

### Train the final model on the training data set

Use the same model as above on the training data split to see how it performs on the training split of the data.

In [28]:
### Train final model on training dataset ###

# Pad the input
tmp_x = pad(preproc_english_sentences_train, max_french_sequence_length_train)

# Train the model
final_model_train = model_final(tmp_x.shape,
                                max_french_sequence_length_train,
                                english_vocab_size_train + 1,
                                french_vocab_size_train + 1)
# Fit the model
final_model_train.fit(tmp_x, preproc_french_sentences_train, batch_size=1024, epochs=10, validation_split=0.2)

# Print out the predictions
print(logits_to_text(final_model_train.predict(tmp_x[:1])[0], french_tokenizer_train))

Epoch 1/10
87/87 [==============================] - 377s 4s/step - loss: 2.2648 - accuracy: 0.5050 - val_loss: 1.4423 - val_accuracy: 0.6239
Epoch 2/10
87/87 [==============================] - 376s 4s/step - loss: 1.2024 - accuracy: 0.6740 - val_loss: 0.9701 - val_accuracy: 0.7281
Epoch 3/10
87/87 [==============================] - 391s 4s/step - loss: 0.8535 - accuracy: 0.7536 - val_loss: 0.7717 - val_accuracy: 0.7695
Epoch 4/10
87/87 [==============================] - 412s 5s/step - loss: 0.6199 - accuracy: 0.8140 - val_loss: 0.5123 - val_accuracy: 0.8457
Epoch 5/10
87/87 [==============================] - 394s 5s/step - loss: 0.4370 - accuracy: 0.8703 - val_loss: 0.4654 - val_accuracy: 0.8602
Epoch 6/10
87/87 [==============================] - 377s 4s/step - loss: 0.3058 - accuracy: 0.9120 - val_loss: 0.3200 - val_accuracy: 0.9051
Epoch 7/10
87/87 [==============================] - 371s 4s/step - loss: 0.2312 - accuracy: 0.9342 - val_loss: 0.3060 - val_accuracy: 0.9094
Epoch 8/10
87

### Test the final model on the test data set

See how the model trained on the training set data performs on the unseen test data.

In [29]:
### Evaluate the final_model trained on the training set on the test data set ###

# Pad the test data input
test_x = pad(preproc_english_sentences_test, max_french_sequence_length_test)
test_x = test_x.reshape((-1, preproc_french_sentences_train.shape[-2]))
preproc_french_sentences_test = preproc_french_sentences_test.reshape((-1, preproc_french_sentences_train.shape[-2], 1))

print(test_x.shape)
print(preproc_french_sentences_train.shape)
print(preproc_french_sentences_test.shape)

# Evaluate the model
final_model_test_score = final_model_train.evaluate(test_x, preproc_french_sentences_test, verbose=0)

print("Final model accuracy on test dataset: {0:.2f}%".format(final_model_test_score[1]*100))

(26260, 21)
(110288, 21, 1)
(26260, 21, 1)
Final model accuracy on test dataset: 20.87%


### Model Performance on Test Data

How did each of the models perform on the test dataset?

In [30]:
print("Simple model accuracy on unseen test dataset: {0:.2f}%".format(simple_rnn_model_score[1]*100))
print("Embedding model accuracy on unseen test data: {0:.2f}%".format(embedding_model_score[1]*100))
print("Bidirectional model accuracy on test data: {0:.2f}%".format(bidirectional_model_score[1]*100))
print("Encoder-decoder model accuracy on the unseen test data: {0:.2f}%".format(encoder_decoder_model_score[1]*100))
print("Final model accuracy on test dataset: {0:.2f}%".format(final_model_test_score[1]*100))

Simple model accuracy on unseen test dataset: 39.36%
Embedding model accuracy on unseen test data: 38.70%
Bidirectional model accuracy on test data: 22.15%
Encoder-decoder model accuracy on the unseen test data: 23.06%
Final model accuracy on test dataset: 20.87%


The Embedding model achieved the highest accuracy on the unseen test data.

It seems none of the models were able to achieve an accuracy level on the test set close to what they achieved on the whole dataset. 

This may be an issue with the `train_test_split` implementation or the lack of data in the training set for the complex models to train on. 

The more complex models (`final_model`, `encoder-decoder` and `bidrectional`) achieved good results on the training data but poor results on the test data. This leads me to believe the models are overfitting on the training set and in turn, not performing well on the test set. 

A next step would be to try and use a larger train/test data split with the complex models to see if their performance improves. 